## Periodic boundary conditions

    * Using symmetry factors to reduce size of linear system
    * One particle in periodic box

In [1]:
from symengine import *
import sympy
import numpy as np
import re
from joblib import Parallel, delayed
from joblib import parallel_backend
from joblib import wrap_non_picklable_objects
import sys
print(sys.executable)

/store/SOFT/gt369/Software/anaconda3/envs/pystokes/bin/python


Number of periodic boxes and Fourier modes

In [24]:
Nb = 1
Nm = 4

Nbb = 2*Nb +1
N1 = int(-(Nm/2)+1) 
N2 = int((Nm/2)+1)

In [25]:
print('side-length of real-space cube: ',Nbb)
print('\nmultiples of wave vector k0 in use in Fourier space:')
for ii in range(N1,N2):
    print(ii)

side-length of real-space cube:  3

multiples of wave vector k0 in use in Fourier space:
-1
0
1
2


### Definitions and auxiliary functions

In [4]:
xij, yij, zij = symbols("xij, yij, zij", real=True) 
b, eta = symbols("b, eta", real=True, positive=True)

# indices for summation:
## Caution, cannot use lambda, instead use Lambda
## do not use eta as index, it's the viscosity
alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma = symbols(
    'alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma', integer=True)

#new symbols for periodic calculation: boxSize, xi (convergence parameter)
L, xi = symbols("L, xi", real=True, positive=True)
kx, ky, kz = symbols("kx, ky, kz", real=True) 

rij = Matrix([xij, yij, zij])
k = Matrix([kx, ky, kz])
kDOTrij = xij*kx + yij*ky + zij*kz
norm_rij = sqrt(xij**2 + yij**2 + zij**2)
norm_k = sqrt(kx**2 + ky**2 + kz**2)

k0 = 2*pi/L

def norm(r):
    result=0
    for i in range(3):
        result += r[i]**2
    return sqrt(result)

KroneckerDelta = eye(3)

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

## Theory

The basic equation for a sedimenting lattice of passive spherical particles is the boundary integral 

$$
\boldsymbol{u}(\boldsymbol{x})=-\sum_{p}\int_{S_{p}}\boldsymbol{G}^{o}\cdot\boldsymbol{f}dS-\int_{\Gamma}\boldsymbol{G}^{o}\cdot\boldsymbol{f}dS,\qquad\boldsymbol{x}\in V,
$$

where $S_{p}$ is the surface of the particle p and \Gamma is a macroscopic surface. Following O'Brien in writing the second integral as

$$
\int_{\Gamma}\boldsymbol{G}^{o}\cdot\left\langle \boldsymbol{f}\right\rangle dS,
$$

labelling it the 'back-flow' integral and assuming, according to Brady, that it will cancel the $\boldsymbol{k}=0$ contribution, we ignore this term and the $\boldsymbol{k}=0$ contribution in the following. Using Ewald summation, we write the Green's function as a periodic Green's function 

$$
\boldsymbol{G}^{P}=\sum_{\boldsymbol{r}}\boldsymbol{M}^{(1)}(\boldsymbol{r})+\frac{1}{V}\sum_{\boldsymbol{k}\neq0}\boldsymbol{M}^{(2)}(\boldsymbol{k}).
$$

The real space sum is over all surrounding particles, while the Fourier space sum is over all wave vectors of the lattice. The two parts of the Green's function are defined as

\begin{align}
\boldsymbol{M}^{(1)}(\boldsymbol{r})&=\frac{1}{8\pi\eta}\left(A\left(\xi r\right)\frac{\boldsymbol{I}}{r}+B\left(\xi r\right)\frac{\boldsymbol{r}\boldsymbol{r}}{r^{3}}\right), \\
\boldsymbol{M}^{(2)}(\boldsymbol{k})&=\frac{C(k)}{\eta}\left(\frac{\boldsymbol{I}}{k^{2}}-\frac{\boldsymbol{k}\boldsymbol{k}}{k^{4}}\right)\cos\left(\boldsymbol{k}\cdot\boldsymbol{r}_{0}\right),
\end{align}

where 

\begin{align}
A(x)&=\text{erfc}(x)+\frac{2}{\sqrt{\pi}}\left(2x^{3}-3x\right)\exp\left(-x^{2}\right), \\
B(x)&=\text{erfc}(x)+\frac{2}{\sqrt{\pi}}\left(x-2x^{3}\right)\exp\left(-x^{2}\right), \\
C(k)&=\left(1+\tfrac{1}{4}\tfrac{k^{2}}{\xi^{2}}+\tfrac{1}{8}\tfrac{k^{4}}{\xi^{4}}\right)\exp\left(-\tfrac{1}{4}\tfrac{k^{2}}{\xi^{2}}\right), \\
\end{align}

and $\boldsymbol{r}_{0}=0$ after evaluation of the matrix elements. This means that for the boundary integral equation for particle $i$ we have

\begin{align}
\boldsymbol{u}_{i}(\boldsymbol{x})&=-\sum_{j}\int_{S_{j}}\boldsymbol{G}_{ij}^{P}\cdot\boldsymbol{f}_{j}dS \\
&=-\int_{S_{i}}\boldsymbol{G}_{ii}^{o}\cdot\boldsymbol{f}_{i}dS-\sum_{\boldsymbol{r}_{ij}}^{\prime}\int_{S_{j}}\boldsymbol{M}^{(1)}(\boldsymbol{r}_{ij})\cdot\boldsymbol{f}_{j}dS-\frac{1}{V}\sum_{\boldsymbol{k}\neq0}\int_{S_{j}}\boldsymbol{M}^{(2)}(\boldsymbol{k})\cdot\boldsymbol{f}_{j}dS+\int_{S_{j}}\boldsymbol{M}^{(2)}(\boldsymbol{r}_{ij}=0)\cdot\boldsymbol{f}_{j}dS. \\
&=-\int_{S_{i}}\boldsymbol{G}_{ii}^{o}\cdot\boldsymbol{f}_{i}dS-\int_{S_{j}}\boldsymbol{G}_{ij}^{*}\cdot\boldsymbol{f}_{j}dS
\end{align}
    
The first term on the RHS is the self-interaction; the prime over the sum in the second term implies $i=j$ is not included and we sum over the periodic boxes; the Fourier space sum in the first term is over all possible wave vectors of the lattice, obeying $r_{i}\cdot k_{j}=2\pi\delta_{ij}$. The lattice vectors are multiples of the box-size $L$ in each direction, and the volume of the periodic cell is $L^{3}$. So the wave vectors are multiples of $2\pi/L$ in each direction of the reciprocal lattice. 

Here it is important to notice that because the Green's function is set up as a periodic Green's function, the above problem actually is a 1-body problem. How does one write that down more clearly?



From hereon we expand the surface fields $\boldsymbol{u}_{i}$ and $\boldsymbol{f}_{i}$ as usual to end up with the linear system 

\begin{align}
\boldsymbol{V}^{L}&=-\boldsymbol{\mathcal{G}}^{LL}\boldsymbol{F}^{L}-\boldsymbol{\mathcal{G}}^{LH}\boldsymbol{F}^{H}+\boldsymbol{\mathcal{K}}^{LH}\boldsymbol{V}^{H}, \\
\frac{1}{2}\boldsymbol{V}^{H}&=-\boldsymbol{\mathcal{G}}^{HL}\boldsymbol{F}^{L}-\boldsymbol{\mathcal{G}}^{HH}\boldsymbol{F}^{H}+\boldsymbol{\mathcal{K}}^{HH}\boldsymbol{V}^{H},
\end{align}

just with the Green's function replaced by

$$
\boldsymbol{G}\rightarrow\boldsymbol{G}^{o}+\underbrace{\sum_{\boldsymbol{r}}^{\prime}\underbrace{\boldsymbol{M}^{(1)}(\boldsymbol{r})}_{\boldsymbol{G}^{R}}}_{\boldsymbol{G}_{\text{total}}^{R}}+\frac{1}{V}\underbrace{\sum_{\boldsymbol{k}\neq0}\underbrace{\boldsymbol{M}^{(2)}(\boldsymbol{k})}_{\boldsymbol{G}^{F}}}_{\boldsymbol{G}_{\text{total}}^{F}}-\boldsymbol{M}^{(2)}(\boldsymbol{r}=0).
$$

### Green's function

First, a function is defined for the Green's function, taking its two cartesian indices.

In [5]:
##pass xi*norm(r) and norm(k) for A, B and C

def A(x): ##are these correct? Don't match Pozrikidis
    return erfc(x) + 2/sqrt(pi)*(2*x**3 - 3*x)*exp(-x**2)

def B(x):
    return erfc(x) + 2/sqrt(pi)*(x - 2*x**3)*exp(-x**2)

def C(k):
    return (S(1) + k**2/xi**2/4 + k**4/xi**4/8)*exp(-k**2/xi**2/4)  ## was exp(-1/4*k**4/xi**4) as in Rajesh' PhD thesis

##define function for periodic Green's function, evaluate it once and save it as another function, as to not have to evaluate it every time 
def GreenReal(alpha, beta): ##eventually evaluate xij=L
    return A(xi*norm_rij)*KroneckerDelta[alpha,beta]/norm_rij + B(xi*norm_rij)*rij[alpha]*rij[beta]/norm_rij**3


def GreenFourier(alpha,beta): ##eventually evaluate xij=0 and take real part
    return 1/eta*C(norm_k)*(KroneckerDelta[alpha,beta]/norm_k**2 - k[alpha]*k[beta]/norm_k**4)*cos(kDOTrij) ## Pozrikidis (3.14)

This function is evaluated to fill a $3\times3$ matrix, which is the Oseen tensor. We keep the real and Fourier part separate for now and add them later.

In [6]:
matGR = zeros(3,3)
for i in range(3):
    for j in range(3):
        matGR[i,j] = GreenReal(i,j)
        
matGF = zeros(3,3)
for i in range(3):
    for j in range(3):
        matGF[i,j] = GreenFourier(i,j)
        
def GR(alpha, beta):
    return matGR[alpha,beta]

def GF(alpha,beta):
    return matGF[alpha,beta]

### Calculus auxiliary functions

In [7]:
def Laplacian(G):
    return diff(G, rij[0], rij[0]) + diff(G, rij[1], rij[1]) + diff(G, rij[2], rij[2])

def Curl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(alpha,i,j)*diff(G(j,beta),rij[i])
    return result

def tilde_Curl(G, alpha, beta): ##introduce for convenience, minus sign already taken into account in matrix elements
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(beta,i,j)*diff(G(alpha, j),rij[i])
    return result

def CurlCurl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(alpha,i,j)*eps(beta,k,l)*diff(G(j,l), rij[i], rij[k])
    return result


def SumG(G, sigma, eta2, mu):
    result=0
    for i in range(3):
        result += eps(sigma, eta2, i)*Laplacian( G(mu, i) )
    return result


def SumG3s1Sym(G, mu, nu2, alpha, gamma1, gamma2):
    result=0
    for i in range(3):
        result += eps(mu,nu2,i)*g3s1Sym(G, alpha,gamma1,gamma2,i) ##add argument G
    return result

In [8]:
##auxiliary functions
def delG(G, alpha, beta, gamma):
    return diff( G(alpha,beta), rij[gamma])

def lapG(G, alpha,beta):
    return Laplacian( G(alpha,beta) )

def dellapG(G, alpha, beta, kappa1):
    return diff(lapG(G, alpha, beta), rij[kappa1])

def deldelG(G, zeta, beta, kappa1, nu):
    return diff(delG(G, zeta, beta, kappa1), rij[nu])

def deldellapG(G, alpha, beta, gamma1, kappa1):
    return diff(dellapG(G, alpha,beta, gamma1), rij[kappa1])

def deldeldelG(G, alpha, beta, gamma1, gamma2, gamma3):
    return diff(deldelG(G, alpha, beta, gamma1, gamma2), rij[gamma3])

def deldeldeldelG(G, alpha, beta, gamma1, gamma2, gamma3, gamma4):
    return diff(deldeldelG(G, alpha, beta, gamma1, gamma2, gamma3), rij[gamma4])

We define the matrix elements 

$$
\boldsymbol{\mathcal{G}}^{(l,l^{\prime})}=\boldsymbol{\mathcal{G}}^{o(l,l^{\prime})}+\boldsymbol{\mathcal{G}}^{*(l,l^{\prime})}=\boldsymbol{\mathcal{G}}^{o(l,l^{\prime})}+\left.b^{l+l^{\prime}-2}\mathcal{F}^{l-1}\tilde{\mathcal{F}}^{l^{\prime}-1}\boldsymbol{\nabla}^{(l-1)}\tilde{\boldsymbol{\nabla}}^{(l^{\prime}-1)}\boldsymbol{G}^{*}(\boldsymbol{R},\tilde{\boldsymbol{R}})\right|_{\boldsymbol{R}=\tilde{\boldsymbol{R}}},
$$

and using the decomposition and projection operators, define the 'irreducible matrix elements. The double-layer matrix elements are computed further down, but for passive particles do not play a role anyway. For the translationally invariant Green's function we use 

$$
\tilde{\boldsymbol{\nabla}}=-\boldsymbol{\nabla}
$$

In [9]:
# GLL  ##simplify later, when filling matrix elements, after subsituting for xij -- much faster
def G1s1s(G, alpha, beta):
    return G(alpha, beta) + b**2/3*lapG(G, alpha, beta)

def G1s2a(G, alpha, beta):
    return -b/2*tilde_Curl(G, alpha,beta)

def G2a1s(G, alpha, beta):
    return b*Curl(G, alpha,beta)

def G2a2a(G, alpha, beta):
    return -b**2/2*CurlCurl(G, alpha,beta)

In [10]:
## GL2s and G2sL
def G1s2s(G, alpha, kappa1, beta):
    return -b/2*((delG(G, alpha, beta, kappa1) + delG(G, alpha, kappa1, beta)) + 4*b**2/15*(dellapG(G, alpha, beta, kappa1) + dellapG(G, alpha, kappa1, beta)))

def G2a2s(G, alpha, kappa1, beta):
    return -b**2/2*(diff(Curl(G, alpha, beta),rij[kappa1]) + diff(Curl(G, alpha, kappa1),rij[beta]))

def G2s1s(G, alpha, gamma1, beta):
    return b/2*((delG(G, alpha, beta, gamma1) + delG(G, gamma1, beta, alpha)) + 4*b**2/15*(dellapG(G, alpha,beta,gamma1) + dellapG(G, gamma1, beta, alpha)))

def G2s2a(G, alpha, gamma1, mu):
    return -b**2/4*(diff(tilde_Curl(G, gamma1,mu), rij[alpha]) + diff(tilde_Curl(G, alpha,mu), rij[gamma1])) #sign change from -> eps(mu, kappa1, beta)


## G2s2s
def G2s2s(G, alpha, gamma1, kappa1, beta):
    return -b**2/4*((deldelG(G, alpha, beta, gamma1, kappa1) + deldelG(G, gamma1, beta, alpha, kappa1)) + 
                      (deldelG(G, alpha,kappa1, gamma1, beta) + deldelG(G, gamma1, kappa1, alpha, beta)) +
                      b**2/5*(deldellapG(G, alpha,beta,gamma1,kappa1) + deldellapG(G, gamma1,beta,alpha,kappa1)) + 
                      b**2/5*(deldellapG(G, alpha, kappa1, gamma1, beta) + deldellapG(G, gamma1,kappa1,alpha,beta)))


## higher matrix elements
def G1s3t(G, alpha, beta):
    return -b**2/5*lapG(G, alpha,beta)


# symmetry for matrix elements of form l sigma, 3s
def g13sSym(G, alpha,kappa2,kappa1,beta):
    return (deldelG(G, alpha,beta,kappa1,kappa2) + deldelG(G, alpha,kappa2,kappa1,beta) + deldelG(G, alpha,kappa1,beta,kappa2))/3 - (
        KroneckerDelta[kappa1,kappa2]*lapG(G, alpha,beta) + KroneckerDelta[kappa1,beta]*lapG(G, alpha,kappa2) + KroneckerDelta[beta,kappa2]*lapG(G, alpha,kappa1))/15

def G1s3s(G, alpha,kappa2, kappa1, beta):
    return b**2*(g13sSym(G, alpha,kappa2,kappa1,beta) + 5*b**2/21*Laplacian(g13sSym(G, alpha,kappa2,kappa1,beta)))


# symmetry of matrix elements of form 3s, l sigma
def g3s1Sym(G, alpha,gamma1,gamma2,beta):
    return (deldelG(G, alpha,beta,gamma1,gamma2) + deldelG(G, gamma1,beta,gamma2,alpha) + deldelG(G, gamma2,beta,alpha,gamma1))/3 - (
        KroneckerDelta[gamma1, gamma2]*lapG(G, alpha,beta) + KroneckerDelta[gamma2,alpha]*lapG(G, gamma1,beta) + KroneckerDelta[alpha,gamma1]*lapG(G, gamma2,beta))/15

def G3s1s(G, alpha,gamma1,gamma2,beta):
    return b**2*(g3s1Sym(G, alpha,gamma1,gamma2,beta) + 5*b**2/21*Laplacian(g3s1Sym(G, alpha,gamma1,gamma2,beta)))

def G3a1s(G, Lambda, kappa2, beta):
    return b**2/2*(diff(Curl(G, Lambda, beta), rij[kappa2]) + diff(Curl(G, kappa2,beta), rij[Lambda]))

def G1s3a(G, alpha, nu2, sigma):
    return 2*b**2/3*(diff(tilde_Curl(G, alpha,nu2),rij[sigma]) - SumG(G, nu2,sigma,alpha)/3) # sign change due to --> eps(sigma,nu1,beta)


def g3sCurlSym(G, alpha,gamma1,gamma2,mu):
    return (diff(tilde_Curl(G, alpha,mu), rij[gamma1],rij[gamma2]) + diff(tilde_Curl(G, gamma1,mu), rij[gamma2],rij[alpha]) + diff(tilde_Curl(G, gamma2,mu), rij[alpha],rij[gamma1]))/3
        
def G3s2a(G, alpha,gamma1,gamma2,mu):
    return -b**3/2*g3sCurlSym(G, alpha, gamma1, gamma2, mu)## sign change due to -> eps(mu, kappa1, beta)

def G3a2a(G, Lambda, kappa2, mu):
    return -b**3/4*(diff(CurlCurl(G, Lambda,mu), rij[kappa2]) + diff(CurlCurl(G, kappa2,mu), rij[Lambda])) #sign change -> eps(mu,kappa1,beta)

def G2s3t(G, alpha,gamma1,nu2):
    return -b**3/10*(dellapG(G, alpha,nu2,gamma1) + dellapG(G, gamma1,nu2,alpha))

def G3s3t(G, alpha,gamma1,gamma2,nu2):
    return -b**4/5*Laplacian(g3s1Sym(G, alpha,gamma1,gamma2,nu2))

def G2s3a(G, alpha,gamma1,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return b**3/3*( diff(tilde_Curl(G, alpha,mu), rij[gamma1], rij[nu2]) - diff(SumG(G, mu,nu2,alpha), rij[gamma1])/3
                   + diff(tilde_Curl(G, gamma1,mu), rij[alpha], rij[nu2]) - diff(SumG(G, mu,nu2,gamma1), rij[alpha])/3)

def G3a3a(G, Lambda, kappa2, mu, nu2): #sign change -> eps(mu,nu1,beta)
    return b**4/3*(diff(CurlCurl(G, Lambda,mu), rij[nu2], rij[kappa2]) + diff(CurlCurl(G, kappa2,mu), rij[nu2], rij[Lambda]))

def G3s3a(G, alpha,gamma1,gamma2,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return 2*b**4/3*(diff(g3sCurlSym(G, alpha,gamma1,gamma2,mu), rij[nu2]) - Laplacian(SumG3s1Sym(G, mu,nu2,alpha,gamma1,gamma2))/3)


def g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta):
    return diff(g13sSym(G, alpha,kappa2,kappa1,beta), rij[gamma1]) + diff(g13sSym(G, gamma1, kappa2, kappa1, beta), rij[alpha])

def G2s3s(G, alpha,gamma1,kappa2, kappa1, beta):
    return b**3/2*(g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta) + 6*b**2/35*Laplacian(g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta)))


def gCurl3sSym(G, tau, kappa2, kappa1, beta):
    return (diff(Curl(G, tau, beta), rij[kappa1],rij[kappa2]) + diff(Curl(G, tau,kappa1), rij[kappa2],rij[beta]) + diff(Curl(G, tau,kappa2), rij[beta],rij[kappa1]))/3 

def G3a3s(G, Lambda, eta2, kappa2, kappa1, beta):
    return b**4/2*(diff(gCurl3sSym(G, Lambda, kappa1, kappa2, beta), rij[eta2]) + diff(gCurl3sSym(G, eta2, kappa1, kappa2, beta), rij[Lambda]))


def deldelg13sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return diff(g13sSym(G, alpha,kappa2,kappa1,beta), rij[gamma1], rij[gamma2])

def lapg13sSym(G, alpha,kappa2,kappa1,beta):
    return Laplacian(g13sSym(G, alpha,kappa2,kappa1,beta))

def g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return (deldelg13sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta) + deldelg13sSym(G, gamma1,gamma2,alpha,kappa2,kappa1,beta) + 
                deldelg13sSym(G, gamma2,alpha,gamma1,kappa2,kappa1,beta))/3 - (
                KroneckerDelta[gamma1,gamma2]*lapg13sSym(G, alpha, kappa2, kappa1, beta) + 
                KroneckerDelta[gamma2,alpha]*lapg13sSym(G, gamma1, kappa2, kappa1, beta) + 
                KroneckerDelta[alpha,gamma1]*lapg13sSym(G, gamma2, kappa2, kappa1, beta))/15

def G3s3s(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return b**4*(g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta) + b**2/7*Laplacian(g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta)))

def G3a2s(G, Lambda, kappa2, kappa1, beta):
    return -b**3/4*(diff(Curl(G, Lambda,beta), rij[kappa1], rij[kappa2]) + diff(Curl(G, Lambda,kappa1), rij[beta], rij[kappa2])
                       + diff(Curl(G, kappa2,beta), rij[kappa1], rij[Lambda]) + diff(Curl(G, kappa2,kappa1), rij[beta], rij[Lambda]))


def g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta):
    return diff(g3s1Sym(G, alpha,gamma1,gamma2,beta), rij[kappa1]) + diff(g3s1Sym(G, alpha,gamma1,gamma2,kappa1), rij[beta])

def G3s2s(G, alpha,gamma1,gamma2,kappa1,beta):
    return -b**3/2*(g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta) + 6*b**2/35*Laplacian(g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta)))

def G2a3a(G, Lambda, mu, nu2): # sign change -> eps(mu,nu1,beta)
    return 2*b**3/3*diff(CurlCurl(G, Lambda, mu), rij[nu2])

def G2a3s(G, Lambda,kappa2,kappa1,beta):
    return b**3*gCurl3sSym(G, Lambda, kappa1, kappa2, beta)

In [11]:
## zero elements
def G3t1s(G, alpha,beta):
    return S(0)

def G3t2a(G, alpha,beta):
    return S(0)

def G3t2s(G, alpha, beta, kappa1):
    return S(0)

def G3t3t(G, alpha,beta):
    return S(0)

def G3t3a(G, alpha, beta, kappa1):
    return S(0)

def G3t3s(G, alpha, beta, kappa1, kappa2):
    return S(0)

def G2a3t(G, alpha,beta):
    return S(0)

def G3a3t(G, alpha, gamma1, beta):
    return S(0)

## Matrix assembly

First, we create matrices from the matrix elements such as 

$$
\boldsymbol{\mathcal{G}}_{5\times5}^{(2s,2s)}.
$$

We do this first for the Fourier part, say 

$$
\boldsymbol{\mathcal{G}}_{5\times5}^{F(2s,2s)}
$$

and then for the real part

$$
\boldsymbol{\mathcal{G}}_{5\times5}^{R(2s,2s)}.
$$

In [12]:
@delayed
@wrap_non_picklable_objects
def matG(G, *args):
    return G(*args)

In [13]:
n_jobs = 50 

In [14]:
# modes = ['1s', '2a', '2s', '3t', '3a', '3s']
modes = ['3s', '2s', '3a', '1s', '2a', '3t'] #for (slightly) faster parallel work while beehive is quiet 
modesV = ['2s', '3t'] ##possible slip modes, for double-layer matrix elements
SLDL  = ['G']#, 'K']
dim   = {'1s':3, '2a':3, '2s':5, '3t':3, '3a':5, '3s':7}

### 1) Fourier part of Green's function

In [15]:
%%time
sldl = 'G' # single-layer
GRF = 'GF' # Fourier space part of Green's function
with parallel_backend('threading'): ## threading is quicker here: less overhead
    for mode1 in modes:
        for mode2 in modes:
            if mode1 in ['1s', '2a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj) for ki in range(3) 
                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                                            for kj in range(2)
                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                                                for kj in range(2)
                                                                                                                    for lj in range(kj,2)
                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')



            elif mode1 == '3t':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj) for ki in range(3) 
                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                                            for kj in range(2)
                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                                                for kj in range(2)
                                                                                                                    for lj in range(kj,2)
                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')



            elif mode1 in ['2s', '3a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, kj) for ki in range(2) 
                                                                                                            for li in range(ki,3)
                                                                                                                for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, kj, lj) for ki in range(2)
                                                                                                                for li in range(ki,3)
                                                                                                                    for kj in range(2)
                                                                                                                        for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, kj, lj, mj) for ki in range(2)
                                                                                                                    for li in range(ki,3)
                                                                                                                        for kj in range(2)
                                                                                                                            for lj in range(kj,2)
                                                                                                                                for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')



            elif mode1 == '3s':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, mi, kj) for ki in range(2) 
                                                                                                                for li in range(ki,2)
                                                                                                                    for mi in range(li,3)
                                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, mi, kj, lj) for ki in range(2)
                                                                                                                    for li in range(ki,2)
                                                                                                                        for mi in range(li,3)
                                                                                                                            for kj in range(2)
                                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, mi, kj, lj, mj) for ki in range(2)
                                                                                                                        for li in range(ki,2)
                                                                                                                            for mi in range(li,3)
                                                                                                                                for kj in range(2)
                                                                                                                                    for lj in range(kj,2)
                                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

Fourier G3s3s done
Fourier G3s2s done
Fourier G3s3a done
Fourier G3s1s done
Fourier G3s2a done
Fourier G3s3t done
Fourier G2s3s done
Fourier G2s2s done
Fourier G2s3a done
Fourier G2s1s done
Fourier G2s2a done
Fourier G2s3t done
Fourier G3a3s done
Fourier G3a2s done
Fourier G3a3a done
Fourier G3a1s done
Fourier G3a2a done
Fourier G3a3t done
Fourier G1s3s done
Fourier G1s2s done
Fourier G1s3a done
Fourier G1s1s done
Fourier G1s2a done
Fourier G1s3t done
Fourier G2a3s done
Fourier G2a2s done
Fourier G2a3a done
Fourier G2a1s done
Fourier G2a2a done
Fourier G2a3t done
Fourier G3t3s done
Fourier G3t2s done
Fourier G3t3a done
Fourier G3t1s done
Fourier G3t2a done
Fourier G3t3t done
CPU times: user 4.77 s, sys: 422 ms, total: 5.19 s
Wall time: 4.84 s


### 2) real space Green's function

In [16]:
%%time
sldl = 'G' # single-layer
GRF = 'GR' # real part of Green's function
with parallel_backend('threading'):
    for mode1 in modes:
        for mode2 in modes:
            if mode1 in ['1s', '2a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj) for ki in range(3) 
                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                                            for kj in range(2)
                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                                                for kj in range(2)
                                                                                                                    for lj in range(kj,2)
                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')



            elif mode1 == '3t':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj) for ki in range(3) 
                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                                            for kj in range(2)
                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                                                for kj in range(2)
                                                                                                                    for lj in range(kj,2)
                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')



            elif mode1 in ['2s', '3a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, kj) for ki in range(2) 
                                                                                                            for li in range(ki,3)
                                                                                                                for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, kj, lj) for ki in range(2)
                                                                                                                for li in range(ki,3)
                                                                                                                    for kj in range(2)
                                                                                                                        for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, kj, lj, mj) for ki in range(2)
                                                                                                                    for li in range(ki,3)
                                                                                                                        for kj in range(2)
                                                                                                                            for lj in range(kj,2)
                                                                                                                                for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')



            elif mode1 == '3s':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, mi, kj) for ki in range(2) 
                                                                                                                for li in range(ki,2)
                                                                                                                    for mi in range(li,3)
                                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, mi, kj, lj) for ki in range(2)
                                                                                                                    for li in range(ki,2)
                                                                                                                        for mi in range(li,3)
                                                                                                                            for kj in range(2)
                                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, mi, kj, lj, mj) for ki in range(2)
                                                                                                                        for li in range(ki,2)
                                                                                                                            for mi in range(li,3)
                                                                                                                                for kj in range(2)
                                                                                                                                    for lj in range(kj,2)
                                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

real G3s3s done
real G3s2s done
real G3s3a done
real G3s1s done
real G3s2a done
real G3s3t done
real G2s3s done
real G2s2s done
real G2s3a done
real G2s1s done
real G2s2a done
real G2s3t done
real G3a3s done
real G3a2s done
real G3a3a done
real G3a1s done
real G3a2a done
real G3a3t done
real G1s3s done
real G1s2s done
real G1s3a done
real G1s1s done
real G1s2a done
real G1s3t done
real G2a3s done
real G2a2s done
real G2a3a done
real G2a1s done
real G2a2a done
real G2a3t done
real G3t3s done
real G3t2s done
real G3t3a done
real G3t1s done
real G3t2a done
real G3t3t done
CPU times: user 10.6 s, sys: 421 ms, total: 11 s
Wall time: 10.6 s


### Sum matrices over real and Fourier spaces

Next, we sum the calculated real matrix elements over the real lattice 

$$
\boldsymbol{\mathcal{G}}_{\text{total}}^{R(l\sigma,l'\sigma')}=\sum_{\boldsymbol{r}}^{\prime}\boldsymbol{\mathcal{G}}^{R(l\sigma,l'\sigma')},
$$

and the Fourier space matrix elements over the wave vectors 

$$
\boldsymbol{\mathcal{G}}_{\text{\text{total}}}^{F(l\sigma,l'\sigma')}=\sum_{\boldsymbol{k}\neq0}\boldsymbol{\mathcal{G}}^{F(l\sigma,l'\sigma')}.
$$

In [17]:
def addGR(matGR):
    totalGR = zeros(matGR.shape[0], matGR.shape[1])
    for ii in range(Nbb):
        dx = (ii - Nb)*L
        for jj in range(Nbb):
            dy = (jj - Nb)*L
            for kk in range(Nbb):
                dz = (kk - Nb)*L
                if ii==jj==kk==Nb:
                    pass
                else:
                    totalGR += matGR.subs({xij:dx, yij:dy, zij:dz})
    return totalGR

def addGF(matGF):
    totalGF = zeros(matGF.shape[0], matGF.shape[1])
    for ii in range(N1, N2): 
        dkx = k0*ii
        for jj in range(N1, N2):
            dky = k0*jj
            for kk in range(N1, N2):
                dkz = k0*kk
                if dkx != 0 or dky != 0 or dkz != 0:
                    totalGF += matGF.subs({kx:dkx, ky:dky, kz:dkz, xij:0, yij:0, zij:0})
    return totalGF

In [18]:
%%time
GRF = 'GR'
for mode1 in modes:
    for mode2 in modes:
        if mode1 in ['1s', '2a']:
            if mode2 in ['1s', '2a', '3t']:
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])  ##write globals()['add'+GRF] instead
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')



        elif mode1 == '3t':
            if mode2 in ['1s', '2a', '3t']:
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')
                
            elif mode2 =='3s':
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')



        elif mode1 in ['2s', '3a']:
            if mode2 in ['1s', '2a', '3t']:
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')



        elif mode1 == '3s':
            if mode2 in ['1s', '2a', '3t']:
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['total'+GRF+mode1+mode2] = addGR(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

total GR3s3s done
total GR3s2s done
total GR3s3a done
total GR3s1s done
total GR3s2a done
total GR3s3t done
total GR2s3s done
total GR2s2s done
total GR2s3a done
total GR2s1s done
total GR2s2a done
total GR2s3t done
total GR3a3s done
total GR3a2s done
total GR3a3a done
total GR3a1s done
total GR3a2a done
total GR3a3t done
total GR1s3s done
total GR1s2s done
total GR1s3a done
total GR1s1s done
total GR1s2a done
total GR1s3t done
total GR2a3s done
total GR2a2s done
total GR2a3a done
total GR2a1s done
total GR2a2a done
total GR2a3t done
total GR3t3s done
total GR3t2s done
total GR3t3a done
total GR3t1s done
total GR3t2a done
total GR3t3t done
CPU times: user 46.2 s, sys: 525 ms, total: 46.7 s
Wall time: 46.6 s


In [19]:
%%time
GRF = 'GF'
for mode1 in modes:
    for mode2 in modes:
        if mode1 in ['1s', '2a']:
            if mode2 in ['1s', '2a', '3t']:
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')



        elif mode1 == '3t':
            if mode2 in ['1s', '2a', '3t']:
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')
                
            elif mode2 =='3s':
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')



        elif mode1 in ['2s', '3a']:
            if mode2 in ['1s', '2a', '3t']:
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')



        elif mode1 == '3s':
            if mode2 in ['1s', '2a', '3t']:
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['total'+GRF+mode1+mode2] = addGF(globals()['mat'+GRF+mode1+mode2])
                print('total '+GRF+mode1+mode2+' done')

total GF3s3s done
total GF3s2s done
total GF3s3a done
total GF3s1s done
total GF3s2a done
total GF3s3t done
total GF2s3s done
total GF2s2s done
total GF2s3a done
total GF2s1s done
total GF2s2a done
total GF2s3t done
total GF3a3s done
total GF3a2s done
total GF3a3a done
total GF3a1s done
total GF3a2a done
total GF3a3t done
total GF1s3s done
total GF1s2s done
total GF1s3a done
total GF1s1s done
total GF1s2a done
total GF1s3t done
total GF2a3s done
total GF2a2s done
total GF2a3a done
total GF2a1s done
total GF2a2a done
total GF2a3t done
total GF3t3s done
total GF3t2s done
total GF3t3a done
total GF3t1s done
total GF3t2a done
total GF3t3t done
CPU times: user 3.72 s, sys: 3.46 ms, total: 3.72 s
Wall time: 3.71 s


### Simplify sympy expressions

The most time-consuming step is to simplify all elements of all matrix elements. This, however, is necessary, otherwise the expressions can be as large as several MB per matrix.

In [22]:
## do not pass matrix as argument, this will not be picklable. String is okay. 
def simplifyElementG(Gstring, alpha, beta): ## extract an element from the matrix G for parallelisation
    return sympy.powdenest(sympy.simplify(globals()['total'+Gstring][alpha,beta]).expand(power_exp=True), force=True)

Simplify matrices in parallel : Fourier space

In [24]:
%%time
sldl = 'G' # single-layer
GRF = 'GF' # Fourier space part of Green's function
with parallel_backend('multiprocessing'): ## multiprocessing is necessary here for simplification of each element on a different core
    for mode1 in modes:
        for mode2 in modes:
            if mode1 in ['1s', '2a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')



            elif mode1 == '3t':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')



            elif mode1 in ['2s', '3a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')



            elif mode1 == '3s':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('Fourier '+sldl+mode1+mode2+' done')

Fourier G3s3s done
Fourier G3s2s done
Fourier G3s3a done
Fourier G3s1s done
Fourier G3s2a done
Fourier G3s3t done
Fourier G2s3s done
Fourier G2s2s done
Fourier G2s3a done
Fourier G2s1s done
Fourier G2s2a done
Fourier G2s3t done
Fourier G3a3s done
Fourier G3a2s done
Fourier G3a3a done
Fourier G3a1s done
Fourier G3a2a done
Fourier G3a3t done
Fourier G1s3s done
Fourier G1s2s done
Fourier G1s3a done
Fourier G1s1s done
Fourier G1s2a done
Fourier G1s3t done
Fourier G2a3s done
Fourier G2a2s done
Fourier G2a3a done
Fourier G2a1s done
Fourier G2a2a done
Fourier G2a3t done
Fourier G3t3s done
Fourier G3t2s done
Fourier G3t3a done
Fourier G3t1s done
Fourier G3t2a done
Fourier G3t3t done
CPU times: user 4.1 s, sys: 3min 33s, total: 3min 37s
Wall time: 4min 2s


real space

In [ ]:
%%time
sldl = 'G' # single-layer
GRF = 'GR' # real space part of Green's function
with parallel_backend('multiprocessing'): 
    for mode1 in modes:
        for mode2 in modes:
            if mode1 in ['1s', '2a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')



            elif mode1 == '3t':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')



            elif mode1 in ['2s', '3a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')



            elif mode1 == '3s':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simplifyElementG)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print('real '+sldl+mode1+mode2+' done')

### Add Green's functions

Finally, we can add the real and Fourier space matrices

$$
\boldsymbol{\mathcal{G}}_{\text{total}}^{(l\sigma,l'\sigma')}=\boldsymbol{\mathcal{G}}_{\text{total}}^{R(l\sigma,l'\sigma')}+\frac{1}{V}\boldsymbol{\mathcal{G}}_{\text{total}}^{F(l\sigma,l'\sigma')}.
$$

In [ ]:
for mode1 in modes:
    for mode2 in modes:
        globals()['mat'+sldl+mode1+mode2] = 1/(8*pi*eta) * globals()['simplifiedGR'+mode1+mode2] + 1/L**3 * globals()['simplifiedGF'+mode1+mode2]
        #globals()['mat'+sldl+mode1+mode2] = 1/L**3 * globals()['simplifiedGF'+mode1+mode2]    #GF only 
        print(sldl+mode1+mode2+' done')

### Create double-layer matrices

We now create the double-layer matrices by noting 

$$\boldsymbol{\mathcal{K}}^{(l\sigma,2s)}=\frac{8\pi\eta b}{3}\boldsymbol{\mathcal{G}}^{(l\sigma,2s)},\qquad\boldsymbol{\mathcal{K}}^{(l\sigma,3t)}=\frac{6\pi\eta b}{5}\boldsymbol{\mathcal{G}}^{(l\sigma,3t)}.
$$

This only works for a translationally invariant Green's function. (I think to remember).

In [ ]:
for mode1 in modes:
    for mode2 in modesV:
        if mode2 == '2s':
            globals()['matK'+mode1+mode2] = 8*pi*eta*b/3 * globals()['matG'+mode1+mode2]
            print('K'+mode1+mode2+' done')
            
        elif mode2 == '3t':
            globals()['matK'+mode1+mode2] = 6*pi*eta*b/5 * globals()['matG'+mode1+mode2]
            print('K'+mode1+mode2+' done')

### Symmetry factors - now it gets tricky to explain

### Delta tensors

We define the delta tensors $\boldsymbol{\Delta}^{(l)}$ just as we defined the Green's function and Oseen tensor earlier. We first define them as a function of their cartesian indices.

In [20]:
## Delta tensors as 'identity' matrices
def Delta1(alpha, beta):
    return KroneckerDelta[alpha,beta]

def Delta2(alpha, gamma1, beta, kappa1):
    return (KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa1]/2
            + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,beta]/2
            -KroneckerDelta[alpha,gamma1]*KroneckerDelta[beta,kappa1]/3)

def Delta3(alpha, gamma1, gamma2, beta, kappa1, kappa2):
    return (KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa1]*KroneckerDelta[gamma2,kappa2] 
                + KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa2]*KroneckerDelta[gamma2,kappa1] 
                + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,beta]*KroneckerDelta[gamma2,kappa2] 
                + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,kappa2]*KroneckerDelta[gamma2,beta] 
                + KroneckerDelta[alpha,kappa2]*KroneckerDelta[gamma1,beta]*KroneckerDelta[gamma2,kappa1]
                + KroneckerDelta[alpha,kappa2]*KroneckerDelta[gamma1,kappa1]*KroneckerDelta[gamma2,beta])/6 - (
        (KroneckerDelta[beta,kappa1]*KroneckerDelta[gamma2,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[gamma2,kappa1]
         + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[gamma2,beta])*KroneckerDelta[alpha,gamma1]
        +(KroneckerDelta[beta,kappa1]*KroneckerDelta[alpha,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[alpha,kappa1]
          + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[alpha,beta])*KroneckerDelta[gamma1,gamma2]
        +(KroneckerDelta[beta,kappa1]*KroneckerDelta[gamma1,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[gamma1,kappa1]
          + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[gamma1,beta])*KroneckerDelta[alpha,gamma2])/15

We then fill them in a loop using the functions above

In [21]:
n_jobs = 1

with parallel_backend('multiprocessing'):
    listDelta1 = Parallel(n_jobs=n_jobs)(delayed(Delta1)(ki,kj) for ki in range(3)
                                                                     for kj in range(3))
matDelta1   = sympy.Matrix(listDelta1).reshape(3,3)

##
with parallel_backend('multiprocessing'):
    listDelta2 = Parallel(n_jobs=n_jobs)(delayed(Delta2)(ki,li, kj,lj) for ki in range(2)
                                                                            for li in range(ki,3)
                                                                                for kj in range(2)
                                                                                  for lj in range(kj,3))
matDelta2   = sympy.Matrix(listDelta2).reshape(5,5)

##
with parallel_backend('multiprocessing'):
    listDelta3 = Parallel(n_jobs=n_jobs)(delayed(Delta3)(ki,li,mi, kj,lj,mj) for ki in range(2)
                                                                                for li in range(ki,2)
                                                                                    for mi in range(li,3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,2)
                                                                                                for mj in range(lj,3))
matDelta3   = sympy.Matrix(listDelta3).reshape(7,7)

### Vectors

By multiplying 

$$
\left(\boldsymbol{\Delta}^{(l\sigma)}\right)^{-1}\cdot\boldsymbol{F}^{(l\sigma)}
$$

and analogous for $\boldsymbol{V}^{(l\sigma)}$ we obtain the correct symmetry factors. These have to be taken into account since when e.g. computing $\boldsymbol{\mathcal{G}}^{(1s,2s)}\boldsymbol{F}^{(2s)}$ in the full sized system we have 

$$\boldsymbol{\mathcal{G}}_{3\times9}^{(1s,2s)}\boldsymbol{F}_{9}^{(2s)}\quad\rightarrow\quad\boldsymbol{\mathcal{G}}_{3\times5}^{(1s,2s)}\boldsymbol{F}_{5}^{(2s)}.
$$

So by reducing the system-size using the symmetries of (2s) we are clearly missing some elements in the end-result. For some reason, the inverse of the *reduced(!)* Delta-tensor works. NB: The full Delta tensors are singular and thus cannot be inverted. 

In [22]:
## In python code will be force[i,0], so have to create force as np.zeros([3*Np,1]) object instead of np.zeros([3*Np])
force  = sympy.Matrix(sympy.MatrixSymbol('force',3,1)) ##create an actual Matrix from it, have to pass to 'Matrix' 
torque = sympy.Matrix(sympy.MatrixSymbol('torque',3,1))
VH     = sympy.Matrix(sympy.BlockDiagMatrix(matDelta2.inv(), matDelta1.inv(), matDelta2.inv(), matDelta3.inv()))*sympy.Matrix(sympy.MatrixSymbol('VH',20,1))
FH     = sympy.Matrix(sympy.BlockDiagMatrix(matDelta2.inv(), matDelta1.inv(), matDelta2.inv(), matDelta3.inv()))*sympy.Matrix(sympy.MatrixSymbol('FH',20,1))

In [23]:
FH

Matrix([
[    2*FH[0, 0] + FH[3, 0]],
[               2*FH[1, 0]],
[               2*FH[2, 0]],
[    FH[0, 0] + 2*FH[3, 0]],
[               2*FH[4, 0]],
[                 FH[5, 0]],
[                 FH[6, 0]],
[                 FH[7, 0]],
[   2*FH[8, 0] + FH[11, 0]],
[               2*FH[9, 0]],
[              2*FH[10, 0]],
[   FH[8, 0] + 2*FH[11, 0]],
[              2*FH[12, 0]],
[4*FH[13, 0] + 3*FH[16, 0]],
[6*FH[14, 0] + 3*FH[18, 0]],
[  4*FH[15, 0] + FH[19, 0]],
[3*FH[13, 0] + 6*FH[16, 0]],
[              6*FH[17, 0]],
[3*FH[14, 0] + 4*FH[18, 0]],
[  FH[15, 0] + 4*FH[19, 0]]])

### Create block matrices

We now assemble e.g. the matrix $\boldsymbol{\mathcal{G}}^{HH}$ from its constituent matrices, where $H=\left\{ 2s,3t,3a,3s\right\}$.

In [ ]:
matG1sH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matG1s2s), sympy.Matrix(matG1s3t), sympy.Matrix(matG1s3a), sympy.Matrix(matG1s3s)]))
matG2aH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matG2a2s), sympy.Matrix(matG2a3t), sympy.Matrix(matG2a3a), sympy.Matrix(matG2a3s)]))

matK1sH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matK1s2s), sympy.Matrix(matK1s3t), sympy.Matrix(sympy.ZeroMatrix(3,12))]))
matK2aH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matK2a2s), sympy.Matrix(matK2a3t), sympy.Matrix(sympy.ZeroMatrix(3,12))]))

matGH1s = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matG2s1s)],
                              [sympy.Matrix(matG3t1s)],
                              [sympy.Matrix(matG3a1s)],
                              [sympy.Matrix(matG3s1s)]]))
matGH2a = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matG2s2a)],
                              [sympy.Matrix(matG3t2a)],
                              [sympy.Matrix(matG3a2a)],
                              [sympy.Matrix(matG3s2a)]]))

matGHH = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matG2s2s), sympy.Matrix(matG2s3t), sympy.Matrix(matG2s3a), sympy.Matrix(matG2s3s)],
                             [sympy.Matrix(matG3t2s), sympy.Matrix(matG3t3t), sympy.Matrix(matG3t3a), sympy.Matrix(matG3t3s)],
                             [sympy.Matrix(matG3a2s), sympy.Matrix(matG3a3t), sympy.Matrix(matG3a3a), sympy.Matrix(matG3a3s)],
                             [sympy.Matrix(matG3s2s), sympy.Matrix(matG3s3t), sympy.Matrix(matG3s3a), sympy.Matrix(matG3s3s)]]))

matKHH_nonzero = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matK2s2s), sympy.Matrix(matK2s3t)],
                                     [sympy.Matrix(matK3t2s), sympy.Matrix(matK3t3t)],
                                     [sympy.Matrix(matK3a2s), sympy.Matrix(matK3a3t)],
                                     [sympy.Matrix(matK3s2s), sympy.Matrix(matK3s3t)]]))
matKHH = sympy.Matrix(sympy.BlockMatrix([matKHH_nonzero, sympy.Matrix(sympy.ZeroMatrix(20,12))]))

# self-interaction

g2s = 3/(20*pi*eta*b)
g3t = 1/(2*pi*eta*b)
g3a = 3/(2*pi*eta*b)
g3s = 6/(7*pi*eta*b)

matGoHH = sympy.Matrix(sympy.BlockDiagMatrix(g2s*matDelta2,
                                             g3t*matDelta1,
                                             g3a*matDelta2,
                                             g3s*matDelta3))


halfMinusk2s = S(3)/5 #0.6
halfMinusk3t = S(2)/5 #0.4
halfMinusk3a = S(4)/5 #0.8
halfMinusk3s = S(19)/35

matKoHH = sympy.Matrix(sympy.BlockDiagMatrix(halfMinusk2s*matDelta2,
                                             halfMinusk3t*matDelta1,
                                             halfMinusk3a*matDelta2,
                                             halfMinusk3s*matDelta3))

Next, we multiply these matrices by place-holder vectors, in order to only have to work with vectors eventually, and be able to use a Krylov solver.

In [ ]:
vecGH1sF = matGH1s*force
vecGH2aT = matGH2a*torque

vecKoHHVH = matKoHH*VH
vecKHHVH = matKHH*VH

vecGoHHFH = matGoHH*FH
vecGHHFH = matGHH*FH

vecG1s1sF = sympy.Matrix(matG1s1s)*force
vecG2a1sF = sympy.Matrix(matG2a1s)*force

vecG1s2aT = sympy.Matrix(matG1s2a)*torque
vecG2a2aT = sympy.Matrix(matG2a2a)*torque

vecG1sHFH = matG1sH*FH
vecG2aHFH = matG2aH*FH

vecK1sHVH = matK1sH*VH
vecK2aHVH = matK2aH*VH

_____________

### Write symbolic results to file as reusable python functions

Eventually, we write our results as functions (returning nothing) to a .py file, which can be reused by the krylov solver, solving the linear system above. 

In [ ]:
ME_file = "periodic_"+str(Nb)+"_"+str(N2-N1)+".py"
dimH = 20

with open(ME_file, "w") as text_file:
    print("import numpy as np", file=text_file)
    print("from math import *", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1sF(v, L, xi, b,eta, force):", file=text_file)
    print("    v += np.array({})".format(np.reshape(np.asarray(vecG1s1sF),(3,)).tolist()), file=text_file) ##cannot mulitply list by 0.5/b 
    print("    return\n", file=text_file)
    
    print("def G2a1sF(o, L, xi, b,eta, force):", file=text_file)
    print("    o += 0.5/b*np.array({})".format(np.reshape(np.asarray(vecG2a1sF),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G1s2aT(v, L, xi, b,eta, torque):", file=text_file)
    print("    v += 1./b*np.array({})".format(np.reshape(np.asarray(vecG1s2aT),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G2a2aT(o, L, xi, b,eta, torque):", file=text_file)
    print("    o += 0.5/(b*b)*np.array({})".format(np.reshape(np.asarray(vecG2a2aT),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GH1sF(gh1sf, L, xi, b,eta, force):", file=text_file)
    print("    gh1sf[:] += {}".format(np.reshape(np.asarray(vecGH1sF),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GH2aT(gh2at, L, xi, b,eta, torque):", file=text_file)
    print("    gh2at[:] += {}".format(np.reshape(np.asarray(vecGH2aT),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KoHHVH(khhvh, b,eta, VH):", file=text_file)
    print("    khhvh[:] -= {}".format(np.reshape(np.asarray(vecKoHHVH),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KHHVH(khhvh, L, xi, b,eta, VH):", file=text_file)
    print("    khhvh[:] += {}".format(np.reshape(np.asarray(vecKHHVH),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GoHHFH(ghhfh, b,eta, FH):", file=text_file)
    print("    ghhfh[:] += {}".format(np.reshape(np.asarray(vecGoHHFH),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GHHFH(ghhfh, L, xi, b,eta, FH):", file=text_file)
    print("    ghhfh[:] += {}".format(np.reshape(np.asarray(vecGHHFH),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G1sHFH(v, L, xi, b,eta, FH):", file=text_file)
    print("    v -= np.array({})".format(np.reshape(np.asarray(vecG1sHFH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G2aHFH(o, L, xi, b,eta, FH):", file=text_file)
    print("    o -= 0.5/b*np.array({})".format(np.reshape(np.asarray(vecG2aHFH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def K1sHVH(v, L, xi, b,eta, VH):", file=text_file)
    print("    v += np.array({})".format(np.reshape(np.asarray(vecK1sHVH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def K2aHVH(o, L, xi, b,eta, VH):", file=text_file)
    print("    o += 0.5/b*np.array({})".format(np.reshape(np.asarray(vecK2aHVH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
#filedata = filedata.replace('sqrt', 'numpy.sqrt')

filedata = re.sub("force\[(\d), 0\]", r"force[\1]", filedata)
filedata = re.sub("torque\[(\d), 0\]", r"torque[\1]", filedata)
filedata = re.sub("VH\[(\d||\d\d), 0\]", r"VH[\1]", filedata)
filedata = re.sub("FH\[(\d||\d\d), 0\]", r"FH[\1]", filedata)

with open(ME_file, 'w') as file:
    file.write(filedata)

______________________________________
_______________________

Remembering the form of the Green's function

$$
\boldsymbol{G}\rightarrow\boldsymbol{G}^{o}+\underbrace{\sum_{\boldsymbol{r}}^{\prime}\underbrace{\boldsymbol{M}^{(1)}(\boldsymbol{r})}_{\boldsymbol{G}^{R}}}_{\boldsymbol{G}_{\text{total}}^{R}}+\frac{1}{V}\underbrace{\sum_{\boldsymbol{k}\neq0}\underbrace{\boldsymbol{M}^{(2)}(\boldsymbol{k})}_{\boldsymbol{G}^{F}}}_{\boldsymbol{G}_{\text{total}}^{R}}-\boldsymbol{M}^{(2)}(\boldsymbol{r}=0).
$$

we follow the same procedure for the last term $\boldsymbol{M}^{(2)}(\boldsymbol{r}=0)$ in a different file. This term does not have to be solved over real space or wave vectors, but its limiting behaviour has to be treated with care. 

In [ ]:
##below direct solver via matrices gives exactly the same result

____
___

### Symbolic matrices to file for direct solver

In [ ]:
ME_file = "matrixPeriodic_"+str(Nb)+"_"+str(N2-N1)+".py"

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("from math import *", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1s(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1s1s),(3,3)).tolist()), file=text_file)
    print("def G2a1s(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2a1s),(3,3)).tolist()), file=text_file)
    
    print("def G1s2a(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1s2a),(3,3)).tolist()), file=text_file)
    print("def G2a2a(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2a2a),(3,3)).tolist()), file=text_file)
    
    print("def GH1s(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGH1s),(dimH,3)).tolist()), file=text_file)
    print("def GH2a(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGH2a),(dimH,3)).tolist()), file=text_file)
    
    print("def KoHH(b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matKoHH),(dimH,dimH)).tolist()), file=text_file)
    print("def KHH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matKHH),(dimH,dimH)).tolist()), file=text_file)
    
    print("def GoHH(b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGoHH),(dimH,dimH)).tolist()), file=text_file)
    print("def GHH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGHH),(dimH,dimH)).tolist()), file=text_file)
    
    print("def G1sH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1sH),(3,dimH)).tolist()), file=text_file)
    print("def G2aH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2aH),(3,dimH)).tolist()), file=text_file)
    
    print("def K1sH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matK1sH),(3,dimH)).tolist()), file=text_file)
    print("def K2aH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matK2aH),(3,dimH)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

with open(ME_file, 'w') as file:
    file.write(filedata)